In [33]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
from collections import Counter
import numpy as np
import sys
import matplotlib.pyplot as plt

# LOAD DATA

In [34]:
# df_usr_view_pay = pd.read_csv('dataset/user_view_pay_df_with_more_features.csv', index_col=[0, 1])
df_usr_view_pay = pd.read_csv('dataset/df_view_pay_with_month_week_aqi_weather.csv', index_col=[0, 1])
df_usr_view_pay

view_cnt  pay_cnt  weekday  month  aqi  weather  wind  \
shop_id time_stamp                                                          
1       2015-10-10         0      188        5     10   64      0.0   0.0   
        2015-10-11         0      272        6     10   68      6.0   0.0   
        2015-10-12         0      257        0     10   85      1.0   0.0   
        2015-10-13         0      239        1     10   84      5.0   0.0   
        2015-10-14         0      277        2     10  143      0.0   0.0   
        2015-10-15         0      227        3     10  164      6.0   0.0   
        2015-10-16         0      242        4     10  135      1.0   0.0   
        2015-10-17         0      293        5     10   78      5.0   0.0   
        2015-10-18         0      291        6     10   66      5.0   0.0   
        2015-10-19         0      256        0     10   67      0.0   0.0   
        2015-10-20         0      262        1     10   76      7.0   0.0   
        2015-10-21         0      271        2     10   69      0.0   0.0   
        2015-10-22         0      260        3     10   85      0.0   0.0   
        2015-10-23         0      240        4     10  102      7.0   0.0   
        2015-10-24         0      291        5     10   77      3.0   0.0   
        2015-10-25         0      286        6     10   71      7.0   0.0   
        2015-10-26         0      264        0     10   83     27.0   0.0   
        2015-10-27         0      295        1     10   66      8.0   0.0   
        2015-10-28         0      310        2     10   86     16.0   0.0   
        2015-10-29         0      212        3     10   76      4.0   0.0   
        2015-10-30         0      331        4     10   51      3.0   0.0   
        2015-10-31         0      347        5     10   40     16.0   0.0   
        2015-11-01         0      381        6     11   66      8.0   0.0   
        2015-11-02         0      332        0     11   80      5.0   0.0   
        2015-11-03         0      404        1     11  111      5.0   0.0   
        2015-11-04         0      392        2     11  112      3.0   0.0   
        2015-11-05         0      496        3     11   89      8.0   0.0   
        2015-11-06         0      430        4     11   63     16.0   0.0   
        2015-11-07         0      351        5     11   46      4.0   0.0   
        2015-11-08         0      288        6     11   42      4.0   0.0   
...                      ...      ...      ...    ...  ...      ...   ...   
2000    2016-10-02         4       39        6     10   65      0.0   2.0   
        2016-10-03         3       50        0     10   37      4.0   5.0   
        2016-10-04         8       29        1     10   46      0.0   1.0   
        2016-10-05         3       35        2     10   56      0.0   1.0   
        2016-10-06         4       76        3     10   54      4.0   5.0   
        2016-10-07        11       68        4     10   43      4.0   2.0   
        2016-10-08        16      158        5     10   36      2.0   5.0   
        2016-10-09        24      176        6     10   42      0.0   5.0   
        2016-10-10        35      161        0     10   42      3.0   2.0   
        2016-10-11        24      163        1     10   46      3.0   2.0   
        2016-10-12        33      159        2     10   52      0.0   2.0   
        2016-10-13        35      177        3     10   67      0.0   2.0   
        2016-10-14        14      148        4     10   58      2.0   2.0   
        2016-10-15         9       78        5     10   55      2.0   1.0   
        2016-10-16         4       60        6     10   60      2.0   2.0   
        2016-10-17        16      163        0     10   80      2.0   2.0   
        2016-10-18        18      157        1     10   61      2.0   1.0   
        2016-10-19        14      164        2     10   50      2.0   1.0   
        2016-10-20        35      165        3     10   40      4.0   2.0   
        2016-10-

In [35]:
df_usr_view_pay.columns

Index(['view_cnt', 'pay_cnt', 'weekday', 'month', 'aqi', 'weather', 'wind',
       'tmp'],
      dtype='object')

In [36]:
quant_features = ['view_cnt', 'pay_cnt', 'weekday', 'month', 'aqi', 'weather', 'wind', 'tmp']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = df_usr_view_pay[each].mean(), df_usr_view_pay[each].std()
    scaled_features[each] = [mean, std]
    df_usr_view_pay.loc[:, each] = (df_usr_view_pay[each] - mean)/std
scaled_features

{'aqi': [69.10689907646486, 41.82863241974223],
 'month': [6.743497867987794, 3.1398223895350226],
 'pay_cnt': [108.25559585194213, 130.7345540633734],
 'tmp': [20.06643747988851, 9.000948791616628],
 'view_cnt': [15.710891994111511, 42.47333460898241],
 'weather': [12.96204816186815, 29.02202674421276],
 'weekday': [3.0034805507322524, 2.003086129482419],
 'wind': [1.3992824375980313, 2.446529545336942]}

In [37]:
# df_usr_view_pay = df_usr_view_pay.drop(['time_stamp', 'view_cnt'], axis=1)
df_usr_view_pay

view_cnt   pay_cnt   weekday     month       aqi  \
shop_id time_stamp                                                     
1       2015-10-10 -0.369900  0.609972  0.996722  1.037161 -0.122091   
        2015-10-11 -0.369900  1.252495  1.495951  1.037161 -0.026463   
        2015-10-12 -0.369900  1.137759 -1.499427  1.037161  0.379957   
        2015-10-13 -0.369900  1.000075 -1.000197  1.037161  0.356050   
        2015-10-14 -0.369900  1.290741 -0.500967  1.037161  1.766567   
        2015-10-15 -0.369900  0.908286 -0.001738  1.037161  2.268616   
        2015-10-16 -0.369900  1.023023  0.497492  1.037161  1.575311   
        2015-10-17 -0.369900  1.413126  0.996722  1.037161  0.212608   
        2015-10-18 -0.369900  1.397828  1.495951  1.037161 -0.074277   
        2015-10-19 -0.369900  1.130110 -1.499427  1.037161 -0.050370   
        2015-10-20 -0.369900  1.176004 -1.000197  1.037161  0.164794   
        2015-10-21 -0.369900  1.244846 -0.500967  1.037161 -0.002556   
        2015-10-22 -0.369900  1.160706 -0.001738  1.037161  0.379957   
        2015-10-23 -0.369900  1.007724  0.497492  1.037161  0.786378   
        2015-10-24 -0.369900  1.397828  0.996722  1.037161  0.188701   
        2015-10-25 -0.369900  1.359582  1.495951  1.037161  0.045258   
        2015-10-26 -0.369900  1.191303 -1.499427  1.037161  0.332143   
        2015-10-27 -0.369900  1.428424 -1.000197  1.037161 -0.074277   
        2015-10-28 -0.369900  1.543161 -0.500967  1.037161  0.403865   
        2015-10-29 -0.369900  0.793550 -0.001738  1.037161  0.164794   
        2015-10-30 -0.369900  1.703791  0.497492  1.037161 -0.432883   
        2015-10-31 -0.369900  1.826177  0.996722  1.037161 -0.695861   
        2015-11-01 -0.369900  2.086246  1.495951  1.355651 -0.074277   
        2015-11-02 -0.369900  1.711440 -1.499427  1.355651  0.260422   
        2015-11-03 -0.369900  2.262175 -1.000197  1.355651  1.001541   
        2015-11-04 -0.369900  2.170386 -0.500967  1.355651  1.025448   
        2015-11-05 -0.369900  2.965891 -0.001738  1.355651  0.475586   
        2015-11-06 -0.369900  2.461051  0.497492  1.355651 -0.145998   
        2015-11-07 -0.369900  1.856773  0.996722  1.355651 -0.552418   
        2015-11-08 -0.369900  1.374881  1.495951  1.355651 -0.648047   
...                      ...       ...       ...       ...       ...   
2000    2016-10-02 -0.275723 -0.529742  1.495951  1.037161 -0.098184   
        2016-10-03 -0.299268 -0.445602 -1.499427  1.037161 -0.767582   
        2016-10-04 -0.181547 -0.606233 -1.000197  1.037161 -0.552418   
        2016-10-05 -0.299268 -0.560338 -0.500967  1.037161 -0.313348   
        2016-10-06 -0.275723 -0.246726 -0.001738  1.037161 -0.361162   
        2016-10-07 -0.110914 -0.307919  0.497492  1.037161 -0.624139   
        2016-10-08  0.006807  0.380499  0.996722  1.037161 -0.791489   
        2016-10-09  0.195160  0.518183  1.495951  1.037161 -0.648047   
        2016-10-10  0.454146  0.403447 -1.499427  1.037161 -0.648047   
        2016-10-11  0.195160  0.418745 -1.000197  1.037161 -0.552418   
        2016-10-12  0.407058  0.388148 -0.500967  1.037161 -0.408976   
        2016-10-13  0.454146  0.525832 -0.001738  1.037161 -0.050370   
        2016-10-14 -0.040282  0.304008  0.497492  1.037161 -0.265533   
        2016-10-15 -0.158002 -0.231428  0.996722  1.037161 -0.337255   
        2016-10-16 -0.275723 -0.369111  1.495951  1.037161 -0.217719   
        2016-10-17  0.006807  0.418745 -1.499427  1.037161  0.260422   
        2016-10-18  0.053895  0.372850 -1.000197  1.037161 -0.193812   
        2016-10-19 -0.040282  0.426394 -0.500967  1.037161 -0.456790   
        2016-10-20  0.454146  0.434043 -0.001738  1.037161 -0.695861   
        2016-10-21 -0.063826  0.227518  0.497492  1.037161 -0.863210   
        2016-10-22 -0.228635 -0.323217  0.996722  1.037161 -1.150095   
        2016-10-23 -0.275723 -0.376760  1.495951  1.037161 -0.743675   
        2016-10-24  0.077439  0.411096 -1.499427  1.037161 -

In [38]:
df_shop_info = pd.read_csv('dataset/normed_shop_info_features.csv', index_col=[0])
df_shop_info

,city_idx,location_id,per_pay,score,comment_cnt,shop_level_0,shop_level_1,shop_level_2,cate_1_idx_0,cate_1_idx_1,...,cate_3_idx_34,cate_3_idx_35,cate_3_idx_36,cate_3_idx_37,cate_3_idx_38,cate_3_idx_39,cate_3_idx_40,cate_3_idx_41,cate_3_idx_42,cate_3_idx_43
1,1.192024,0.899196,-0.430197,1.397435e+00,2.742265e+00,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1.402179,-1.545979,1.478704,-1.501043e-14,4.805603e-15,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.403944,0.568606,-0.950806,3.411693e-01,-3.495221e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0.561560,-0.604840,1.305167,-1.501043e-14,4.805603e-15,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,-0.594291,-0.953299,-1.471415,-7.150959e-01,-3.495221e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0.666637,1.655681,0.437485,3.411693e-01,-6.587009e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,-0.016366,-1.304737,0.437485,3.411693e-01,1.814729e+00,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,-0.594291,-1.039670,0.958095,3.411693e-01,-6.587009e-01,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,-0.594291,1.599093,0.784558,1.397435e+00,-9.678796e-01,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
10,1.664872,-1.444717,-0.083124,-1.501043e-14,4.805603e-15,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


# SPLIT DATA TO TWO SETS

In [39]:
validation_ratio = 0.1
num_samples = 2000

In [40]:
validation_idx = np.random.randint(1, num_samples+1, size=int(validation_ratio*num_samples))
print(validation_idx.shape)
validation_idx

(200,)


array([ 427,   50,  558,  100,  531,   42, 1684,  847,  714,  229, 1755,
        638, 1098,  939,  941, 1935,  233, 1419, 1154, 1792,  930, 1137,
       1269,  939,  398, 1519, 1037, 1800,  944, 1471,   57, 1630, 1714,
        718, 1145, 1418, 1614, 1530, 1649,  740, 1289, 1477, 1974,  713,
       1690,  787,  567,  248,  949,  349,  992,  537,  401, 1997, 1395,
       1914, 1768, 1950,  755, 1357,  329, 1880, 1283, 1539, 1705,  139,
        265, 1664, 1716, 1346,  585,  295,  814,  866, 1285,  307, 1460,
        391, 1782,  397,  710,   54, 1255, 1734, 1614, 1354,  985,  462,
       1641,  382,  247,  609,   68,  865,  277,  346, 1405, 1497, 1170,
        731,  548, 1509, 1204, 1891, 1984,  207,   23,  781,  879,  755,
       1483, 1934, 1249, 1279,  734,  205,  418,  616,  259, 1617,  705,
       1170,  693, 1449, 1098,  868,  766,  796, 1028,    2, 1824, 1621,
       1778,  743,  354, 1179, 1841,  476, 1482,  746, 1278,   10,  442,
       1190,  134, 1879, 1031,  802, 1182,  642, 14

In [41]:
len(set(validation_idx))

192

In [42]:
df_usr_train = df_usr_view_pay.drop(list(validation_idx), axis=0)
df_usr_train.reset_index(inplace=True)
df_usr_train.set_index(keys=['shop_id', 'time_stamp'], inplace=True)
df_usr_train

view_cnt   pay_cnt   weekday     month       aqi  \
shop_id time_stamp                                                     
1       2015-10-10 -0.369900  0.609972  0.996722  1.037161 -0.122091   
        2015-10-11 -0.369900  1.252495  1.495951  1.037161 -0.026463   
        2015-10-12 -0.369900  1.137759 -1.499427  1.037161  0.379957   
        2015-10-13 -0.369900  1.000075 -1.000197  1.037161  0.356050   
        2015-10-14 -0.369900  1.290741 -0.500967  1.037161  1.766567   
        2015-10-15 -0.369900  0.908286 -0.001738  1.037161  2.268616   
        2015-10-16 -0.369900  1.023023  0.497492  1.037161  1.575311   
        2015-10-17 -0.369900  1.413126  0.996722  1.037161  0.212608   
        2015-10-18 -0.369900  1.397828  1.495951  1.037161 -0.074277   
        2015-10-19 -0.369900  1.130110 -1.499427  1.037161 -0.050370   
        2015-10-20 -0.369900  1.176004 -1.000197  1.037161  0.164794   
        2015-10-21 -0.369900  1.244846 -0.500967  1.037161 -0.002556   
        2015-10-22 -0.369900  1.160706 -0.001738  1.037161  0.379957   
        2015-10-23 -0.369900  1.007724  0.497492  1.037161  0.786378   
        2015-10-24 -0.369900  1.397828  0.996722  1.037161  0.188701   
        2015-10-25 -0.369900  1.359582  1.495951  1.037161  0.045258   
        2015-10-26 -0.369900  1.191303 -1.499427  1.037161  0.332143   
        2015-10-27 -0.369900  1.428424 -1.000197  1.037161 -0.074277   
        2015-10-28 -0.369900  1.543161 -0.500967  1.037161  0.403865   
        2015-10-29 -0.369900  0.793550 -0.001738  1.037161  0.164794   
        2015-10-30 -0.369900  1.703791  0.497492  1.037161 -0.432883   
        2015-10-31 -0.369900  1.826177  0.996722  1.037161 -0.695861   
        2015-11-01 -0.369900  2.086246  1.495951  1.355651 -0.074277   
        2015-11-02 -0.369900  1.711440 -1.499427  1.355651  0.260422   
        2015-11-03 -0.369900  2.262175 -1.000197  1.355651  1.001541   
        2015-11-04 -0.369900  2.170386 -0.500967  1.355651  1.025448   
        2015-11-05 -0.369900  2.965891 -0.001738  1.355651  0.475586   
        2015-11-06 -0.369900  2.461051  0.497492  1.355651 -0.145998   
        2015-11-07 -0.369900  1.856773  0.996722  1.355651 -0.552418   
        2015-11-08 -0.369900  1.374881  1.495951  1.355651 -0.648047   
...                      ...       ...       ...       ...       ...   
2000    2016-10-02 -0.275723 -0.529742  1.495951  1.037161 -0.098184   
        2016-10-03 -0.299268 -0.445602 -1.499427  1.037161 -0.767582   
        2016-10-04 -0.181547 -0.606233 -1.000197  1.037161 -0.552418   
        2016-10-05 -0.299268 -0.560338 -0.500967  1.037161 -0.313348   
        2016-10-06 -0.275723 -0.246726 -0.001738  1.037161 -0.361162   
        2016-10-07 -0.110914 -0.307919  0.497492  1.037161 -0.624139   
        2016-10-08  0.006807  0.380499  0.996722  1.037161 -0.791489   
        2016-10-09  0.195160  0.518183  1.495951  1.037161 -0.648047   
        2016-10-10  0.454146  0.403447 -1.499427  1.037161 -0.648047   
        2016-10-11  0.195160  0.418745 -1.000197  1.037161 -0.552418   
        2016-10-12  0.407058  0.388148 -0.500967  1.037161 -0.408976   
        2016-10-13  0.454146  0.525832 -0.001738  1.037161 -0.050370   
        2016-10-14 -0.040282  0.304008  0.497492  1.037161 -0.265533   
        2016-10-15 -0.158002 -0.231428  0.996722  1.037161 -0.337255   
        2016-10-16 -0.275723 -0.369111  1.495951  1.037161 -0.217719   
        2016-10-17  0.006807  0.418745 -1.499427  1.037161  0.260422   
        2016-10-18  0.053895  0.372850 -1.000197  1.037161 -0.193812   
        2016-10-19 -0.040282  0.426394 -0.500967  1.037161 -0.456790   
        2016-10-20  0.454146  0.434043 -0.001738  1.037161 -0.695861   
        2016-10-21 -0.063826  0.227518  0.497492  1.037161 -0.863210   
        2016-10-22 -0.228635 -0.323217  0.996722  1.037161 -1.150095   
        2016-10-23 -0.275723 -0.376760  1.495951  1.037161 -0.743675   
        2016-10-24  0.077439  0.411096 -1.499427  1.037161 -

In [43]:
df_usr_train.index.levels[0]

Int64Index([   1,    3,    4,    5,    6,    7,    8,    9,   11,   12,
            ...
            1990, 1991, 1992, 1993, 1994, 1995, 1996, 1998, 1999, 2000],
           dtype='int64', name='shop_id', length=1808)

In [44]:
df_usr_validation = df_usr_view_pay.loc[list(validation_idx)]
df_usr_validation.reset_index(inplace=True)
df_usr_validation.set_index(keys=['shop_id', 'time_stamp'], inplace=True)
df_usr_validation

view_cnt   pay_cnt   weekday     month       aqi  \
shop_id time_stamp                                                     
2       2015-11-25 -0.369900 -0.407357 -0.500967  1.355651  1.144984   
        2015-11-26 -0.369900  0.510534 -0.001738  1.355651  1.312333   
        2015-11-27 -0.369900  0.594674  0.497492  1.355651  1.049355   
        2015-11-28 -0.369900 -0.109042  0.996722  1.355651  1.575311   
        2015-11-29 -0.369900 -0.606233  1.495951  1.355651  2.722850   
        2015-11-30 -0.369900 -0.675075 -1.499427  1.355651  2.675036   
        2015-12-01 -0.369900 -0.667426 -1.000197  1.674140  0.810285   
        2015-12-02 -0.369900 -0.598584 -0.500967  1.674140  0.929820   
        2015-12-03 -0.369900 -0.621531 -0.001738  1.674140  0.523400   
        2015-12-04 -0.369900 -0.667426  0.497492  1.674140 -0.337255   
        2015-12-05 -0.369900 -0.629180  0.996722  1.674140 -0.432883   
        2015-12-06 -0.369900 -0.445602  1.495951  1.674140  0.260422   
        2015-12-07 -0.369900  0.082185 -1.499427  1.674140  4.085553   
        2015-12-08 -0.369900  0.870041 -1.000197  1.674140  1.694846   
        2015-12-09 -0.369900  1.053619 -0.500967  1.674140  4.085553   
        2015-12-10 -0.369900  1.443722 -0.001738  1.674140  2.220802   
        2015-12-11 -0.369900  1.757335  0.497492  1.674140  1.790475   
        2015-12-12 -0.369900 -0.828056  0.996722  1.674140  5.519977   
        2015-12-13 -0.369900 -0.047850  1.495951  1.674140  8.245383   
        2015-12-14 -0.369900 -0.361462 -1.499427  1.674140  6.739238   
        2015-12-15 -0.369900 -0.269673 -1.000197  1.674140  3.081456   
        2015-12-16 -0.369900 -0.262024 -0.500967  1.674140  0.642935   
        2015-12-17 -0.369900 -0.231428 -0.001738  1.674140  0.858099   
        2015-12-18 -0.369900 -0.040201  0.497492  1.674140  1.647032   
        2015-12-19 -0.369900 -0.063148  0.996722  1.674140  1.073262   
        2015-12-20 -0.369900  0.181623  1.495951  1.674140  0.690749   
        2015-12-21 -0.369900 -0.109042 -1.499427  1.674140  2.961921   
        2015-12-22 -0.369900  0.013343 -1.000197  1.674140  2.268616   
        2015-12-23 -0.369900 -0.047850 -0.500967  1.674140  3.655226   
        2015-12-24 -0.369900  0.112781 -0.001738  1.674140  4.372438   
...                      ...       ...       ...       ...       ...   
1997    2016-10-02 -0.322812 -0.315568  1.495951  1.037161 -0.863210   
        2016-10-03 -0.299268 -0.384409 -1.499427  1.037161 -0.624139   
        2016-10-04 -0.346356 -0.361462 -1.000197  1.037161 -0.911024   
        2016-10-05 -0.346356 -0.369111 -0.500967  1.037161 -0.385069   
        2016-10-06 -0.299268 -0.193182 -0.001738  1.037161 -0.600232   
        2016-10-07 -0.205091 -0.262024  0.497492  1.037161 -0.648047   
        2016-10-08 -0.134458  0.854743  0.996722  1.037161 -0.815396   
        2016-10-09 -0.299268  1.351933  1.495951  1.037161 -0.743675   
        2016-10-10 -0.228635  1.459021 -1.499427  1.037161 -0.887117   
        2016-10-11 -0.110914  1.566108 -1.000197  1.037161 -0.911024   
        2016-10-12 -0.087370  1.359582 -0.500967  1.037161 -0.576325   
        2016-10-13 -0.228635  1.351933 -0.001738  1.037161 -0.624139   
        2016-10-14 -0.087370  1.336635  0.497492  1.037161 -0.576325   
        2016-10-15 -0.205091 -0.086095  0.996722  1.037161 -0.743675   
        2016-10-16 -0.205091  0.097483  1.495951  1.037161 -0.002556   
        2016-10-17 -0.252179  1.780282 -1.499427  1.037161 -0.456790   
        2016-10-18 -0.275723  1.596704 -1.000197  1.037161 -0.863210   
        2016-10-19 -0.346356  1.512564 -0.500967  1.037161 -0.456790   
        2016-10-20 -0.252179  1.252495 -0.001738  1.037161 -0.815396   
        2016-10-21 -0.275723  0.862392  0.497492  1.037161 -1.126188   
        2016-10-22 -0.205091 -0.162586  0.996722  1.037161 -1.221816   
        2016-10-23 -0.322812  0.074536  1.495951  1.037161 -0.695861   
        2016-10-24 -0.299268  1.382530 -1.499427  1.037161 -

In [45]:
len(df_usr_validation.index.levels[0])

192

# START NN!

### Define Data Generator

In [46]:
predict_days = 14

In [65]:
class Config(object):
    num_steps = 32
    batch_size = 100
    num_lstm_hidden = 16
    num_lstm_layers = 2
    num_lstm_features = 8
    num_shop_features = 75
#     keep_prob = 0.75
config = Config()

In [66]:
df_usr = df_usr_train


In [67]:
len(list(df_usr.index.levels[0]))

1808

In [68]:
a = [np.array([[1,2],[3,4]]), np.array([[5,6],[7,8]])]
print(a)
b = np.stack(a)
print(b)
b.shape

[array([[1, 2],
       [3, 4]]), array([[5, 6],
       [7, 8]])]
[[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]


(2, 2, 2)

In [69]:
ss = df_usr_view_pay.loc[1]
ss.reset_index(drop=True)

,view_cnt,pay_cnt,weekday,month,aqi,weather,wind,tmp
0,-0.369900,0.609972,0.996722,1.037161,-0.122091,-0.446628,-0.571946,-0.340679
1,-0.369900,1.252495,1.495951,1.037161,-0.026463,-0.239888,-0.571946,-0.396229
2,-0.369900,1.137759,-1.499427,1.037161,0.379957,-0.412171,-0.571946,-0.285130
3,-0.369900,1.000075,-1.000197,1.037161,0.356050,-0.274345,-0.571946,-0.285130
4,-0.369900,1.290741,-0.500967,1.037161,1.766567,-0.446628,-0.571946,-0.229580
5,-0.369900,0.908286,-0.001738,1.037161,2.268616,-0.239888,-0.571946,-0.118481
6,-0.369900,1.023023,0.497492,1.037161,1.575311,-0.412171,-0.571946,0.048169
7,-0.369900,1.413126,0.996722,1.037161,0.212608,-0.274345,-0.571946,-0.007381
8,-0.369900,1.397828,1.495951,1.037161,-0.074277,-0.274345,-0.571946,-0.007381
9,-0.369900,1.130110,-1.499427,1.037161,-0.050370,-0.446628,-0.571946,-0.062931


In [70]:
def data_generator(config, df_usr, df_shop, random=True):
    batch_size = config.batch_size
    num_steps = config.num_steps
    ID_list = list(df_usr.index.levels[0])
    m = len(ID_list)
    assert batch_size <= m, 'batch_size({}) > sample number({}), lower batch_size!'.format(batch_size, m)
    for batch_id in range(int(m/batch_size)): 
        if random:
            np.random.shuffle(ID_list)
            batch_ID_list = ID_list[0:batch_size]
        else:
            batch_ID_list = ID_list[batch_id*batch_size:(batch_id+1)*batch_size]
        _features, _labels = [], []
        _shop_info = []
        for ID in batch_ID_list:
            each_shop_info = df_shop.loc[ID].values
            _shop_info.append(each_shop_info)
            
            each_shop_usr = df_usr.loc[ID]
            each_shop_usr = each_shop_usr.reset_index(drop=True)
            _m = each_shop_usr.shape[0] - predict_days
            assert num_steps < _m, 'lower num_steps!'
            _s_idx = np.random.randint(0, _m-num_steps, 1)[0]
            _e_idx = _s_idx + num_steps
#             print('_m: {}, _s_idx: {}, _e_idx: {}'.format(_m, _s_idx, _e_idx))
            _features.append(each_shop_usr.loc[_s_idx:_e_idx-1].values)
            _labels.append(each_shop_usr.loc[_e_idx:(_e_idx+predict_days-1), 'pay_cnt'].values)
        
#         for label in _labels:
#             print(label.shape)
        
        features = np.stack(_features)
        labels = np.stack(_labels)
        shop_info = np.stack(_shop_info)
#         print('features.shape: {}, labels.shape: {}, shop_info.shape: {}'.format(features.shape, labels.shape, shop_info.shape))
        yield features, labels, shop_info, batch_ID_list


In [71]:
f, l, s, _ = data_generator(config, df_usr_train, df_shop_info).__next__()
l.shape

(100, 14)

### Define LSTM

In [72]:
import tensorflow as tf

In [73]:
tf.__version__

'1.0.0'

In [74]:
def nn_lstm(x_tensor, config, _keep_prob):
    num_steps = config.num_steps
    num_lstm_hidden = config.num_lstm_hidden
    num_lstm_layers = config.num_lstm_layers
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x_tensor = tf.transpose(x_tensor, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x_tensor = tf.reshape(x_tensor, [-1, 1])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x_tensor = tf.split(x_tensor, num_or_size_splits=num_steps, axis=0)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.LSTMCell(num_lstm_hidden)
    lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, input_keep_prob=_keep_prob, output_keep_prob=_keep_prob)
    lstm_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell]*num_lstm_layers)
    

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x_tensor, dtype=tf.float32, sequence_length=None)
#     print('outputs.shape: {}, {}, states.shape: {}'.format(len(outputs),outputs[-1].shape, len(states)))
    return outputs[-1]

In [75]:
def merge(lstm_out, shop_info, config):
    num_lstm_features = config.num_lstm_features
    num_lstm_hidden = config.num_lstm_hidden
    lstm_out = tf.reshape(lstm_out, shape=[-1, num_lstm_features*num_lstm_hidden])
    return tf.concat([lstm_out, shop_info], axis=1)

In [76]:
def fully_conn(x_tensor, num_outputs):
    """
    Apply a fully connected layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    w_fc = tf.Variable(tf.truncated_normal([x_tensor.get_shape().as_list()[1], num_outputs], stddev=0.1))
    b_fc = tf.Variable(tf.zeros(num_outputs))
    fc = tf.add(tf.matmul(x_tensor, w_fc), b_fc)
    fc = tf.nn.relu(fc)
    return fc

In [77]:
def output(x_tensor, num_outputs):
    """
    Apply a output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    w_o = tf.Variable(tf.truncated_normal([x_tensor.get_shape().as_list()[1], num_outputs], stddev=0.5))
    b_o = tf.Variable(tf.zeros(num_outputs))
    output = tf.add(tf.matmul(x_tensor, w_o), b_o)
    return output

In [78]:
def whole_nn(x_seq, x_shop, config, _keep_prob):

    lstm_out = nn_lstm(x_seq, config, _keep_prob)
    
    merge_feature = merge(lstm_out, x_shop, config)
    
    print('merge_feature.shape: {}'.format(merge_feature.shape))
    fc_1 = fully_conn(merge_feature, 1024)
    fc_1 = tf.nn.dropout(fc_1, _keep_prob)
    
    fc_2 = fully_conn(fc_1, 1024)
#     fc_2 = tf.nn.dropout(fc_2, _keep_prob)
    
    fc_3 = fully_conn(fc_2, 1024)
    fc_2 = tf.nn.dropout(fc_2, _keep_prob)
    
    regression_out = output(fc_3, predict_days)
    
    return regression_out

In [79]:
def reg_loss(y_pred, y_g):
    _tmp = tf.abs(tf.div(tf.subtract(y_pred, y_g), tf.add(y_pred, y_g)))
    return tf.reduce_mean(_tmp, name='T_loss')
#     m, n = y_pred.shape
#     _sub = tf.subtract(y_pred, y_g)
#     _add = tf.add(y_pred, y_g)
#     _sum = 0
#     for i in xrange(m):
#         for j in xrange(n):
#             if _add[i, j] == 0:
#                 continue
#             else:
#                 _sum += abs(_sub[i, j] / _add[i, j])
#     return _sum / (m*n)

In [80]:
##############################
## Build the Neural Network ##
##############################

# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x_seq = tf.placeholder(tf.float32, shape=[None, config.num_steps, config.num_lstm_features], name='x_seq')
x_shop = tf.placeholder(tf.float32, shape=[None, config.num_shop_features], name='x_shop')
y = tf.placeholder(tf.float32, shape=[None, predict_days], name='y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# Model
reg_out = whole_nn(x_seq, x_shop, config, keep_prob)

# Name reg_out Tensor, so that is can be loaded from disk after training
reg_out = tf.identity(reg_out, name='reg_out')

# Loss and Optimizer
cost = tf.reduce_mean(tf.pow(tf.subtract(reg_out, y), 2), name='cost')
T_loss = reg_loss(reg_out, y)
optimizer = tf.train.AdamOptimizer().minimize(cost)

# # Accuracy
# correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


merge_feature.shape: (?, 203)


In [81]:
epochs = 50
keep_probablity = 0.75

In [82]:
save_model_path = 'model/customer_prediction'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        for batch_id, (features, labels, shop_info, _) in enumerate(data_generator(config, df_usr_view_pay, df_shop_info)):
            train_feed_dict = {x_seq: features, x_shop: shop_info, y: labels, keep_prob: keep_probablity}
            sess.run(optimizer, feed_dict=train_feed_dict)
            train_cost = sess.run(cost, feed_dict=train_feed_dict)
            train_T_loss = sess.run(T_loss, feed_dict=train_feed_dict)
            print('epoch: {:>2}, batch: {:>2}, train_cost: {:>8.4f}, train_T_loss: {:>8.4f}'.format(
                epoch+1, batch_id+1, train_cost, train_T_loss))
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)


epoch:  1, batch:  1, train_cost: 267.0483, train_T_loss:   1.5475
epoch:  1, batch:  2, train_cost:  83.6268, train_T_loss:   1.4416
epoch:  1, batch:  3, train_cost:  45.5079, train_T_loss:   1.8227
epoch:  1, batch:  4, train_cost:  75.4768, train_T_loss:   4.0012
epoch:  1, batch:  5, train_cost:  85.7582, train_T_loss:   1.4320
epoch:  1, batch:  6, train_cost:  65.5049, train_T_loss:   1.2558
epoch:  1, batch:  7, train_cost:  37.1016, train_T_loss:   1.3473
epoch:  1, batch:  8, train_cost:  23.6364, train_T_loss:   1.9846
epoch:  1, batch:  9, train_cost:  18.8298, train_T_loss:   2.3724
epoch:  1, batch: 10, train_cost:  23.6366, train_T_loss:   2.1579
epoch:  1, batch: 11, train_cost:  25.7166, train_T_loss:   2.1700
epoch:  1, batch: 12, train_cost:  23.8802, train_T_loss:   1.8305
epoch:  1, batch: 13, train_cost:  22.7133, train_T_loss:   6.6260
epoch:  1, batch: 14, train_cost:  24.4292, train_T_loss:   2.7144
epoch:  1, batch: 15, train_cost:  16.7056, train_T_loss:   1.

# CHECKPOINT

In [83]:
def final_data_generator(config, df_usr, df_shop, random=True):
    batch_size = config.batch_size
    num_steps = config.num_steps
    ID_list = list(df_usr.index.levels[0])
    m = len(ID_list)
    assert batch_size <= m, 'batch_size({}) > sample number({}), lower batch_size!'.format(batch_size, m)
    for batch_id in range(int(m/batch_size)): 
        if random:
            np.random.shuffle(ID_list)
            batch_ID_list = ID_list[0:batch_size]
        else:
            batch_ID_list = ID_list[batch_id*batch_size:(batch_id+1)*batch_size]
        _features, _labels = [], []
        _shop_info = []
        for ID in batch_ID_list:
            each_shop_info = df_shop.loc[ID].values
            _shop_info.append(each_shop_info)
            
            each_shop_usr = df_usr.loc[ID]
            each_shop_usr = each_shop_usr.reset_index(drop=True)
            _e_idx = each_shop_usr.shape[0]
            _s_idx = _e_idx - num_steps
#             print('_m: {}, _s_idx: {}, _e_idx: {}'.format(_m, _s_idx, _e_idx))
            _features.append(each_shop_usr.loc[_s_idx:_e_idx-1].values)
        
        features = np.stack(_features)
        shop_info = np.stack(_shop_info)
#         print('features.shape: {}, labels.shape: {}, shop_info.shape: {}'.format(features.shape, labels.shape, shop_info.shape))
        yield features, shop_info, batch_ID_list

In [84]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf

save_model_path = 'model/customer_prediction'

test_config = Config()
test_config.batch_size = 50

"""
Test the saved model against the test dataset
"""
loaded_graph = tf.Graph()

with tf.Session(graph=loaded_graph) as sess:
    # Load model
    loader = tf.train.import_meta_graph(save_model_path + '.meta')
    loader.restore(sess, save_model_path)

    # Get Tensors from loaded model
    loaded_x_seq = loaded_graph.get_tensor_by_name('x_seq:0')
    loaded_x_shop = loaded_graph.get_tensor_by_name('x_shop:0')
    loaded_y = loaded_graph.get_tensor_by_name('y:0')
    loaded_reg_out = loaded_graph.get_tensor_by_name('reg_out:0')
    loaded_cost = loaded_graph.get_tensor_by_name('cost:0')
    loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    # Get accuracy in batches for memory limitations
    test_batch_cost_total = 0
    test_batch_count = 0
    result_list = []
    reg_out_list = []
    for features, labels, shop_info, batch_ID_list in data_generator(test_config, df_usr_view_pay, df_shop_info, random=False):
        test_feed_dict={loaded_x_seq: features, loaded_x_shop: shop_info, loaded_y: labels, loaded_keep_prob: 1.}
        test_batch_cost_total += sess.run(loaded_cost, feed_dict=test_feed_dict)
        test_batch_count += 1
        test_reg_out = sess.run(loaded_reg_out, feed_dict=test_feed_dict)
        tmp_list = [pd.DataFrame(np.array([test_reg_out[i,:], labels[i,:]]).T, columns=['pred', 'label']) for i in range(test_config.batch_size)]
        result_list.append(pd.concat(tmp_list, keys=batch_ID_list, names=['shop_id', 'predict_day']))
        
        reg_out_list.append(pd.DataFrame(test_reg_out))
        
    df_result = pd.concat(result_list)
    
    final_reg_out_list = []
    for features, shop_info, batch_ID_list in final_data_generator(test_config, df_usr_view_pay, df_shop_info, random=False):
        test_feed_dict={loaded_x_seq: features, loaded_x_shop: shop_info, loaded_keep_prob: 1.}
        final_reg_out = sess.run(loaded_reg_out, feed_dict=test_feed_dict)
        final_reg_out_list.append(pd.DataFrame(final_reg_out))
   
    df_pred = pd.concat(final_reg_out_list, ignore_index=True)
    

    print('Testing Cost: {}\n'.format(test_batch_cost_total/test_batch_count))


Testing Cost: 0.2922699611634016



In [85]:
def negtive_check(df, op=True):
    a = unnormed_df_pred < 0
    if a.sum().sum() > 0:
        print('Find negative!')
        if op:
            df[a] = 0
    else:
        print('no negative number~')

In [86]:
df_result

pred     label
shop_id predict_day                    
1       0            0.551392  0.740006
        1            0.915087  0.686463
        2            0.836080  0.556428
        3            0.937462  0.977128
        4            0.530437  0.732357
        5            0.206616  0.915935
        6            1.014682  0.816497
        7            0.428193  1.007724
        8            0.649460  0.388148
        9            0.325520  0.671165
        10           0.730050  0.357552
        11           0.550544  0.365201
        12           0.632398  0.510534
        13           0.748899  0.663516
2       0            0.835268  1.459021
        1            0.774406  0.762954
        2            0.693780 -0.231428
        3            0.682917 -0.338515
        4            0.484057 -0.269673
        5            0.758804 -0.284971
        6            0.459608  0.005694
        7            0.684599  0.066887
        8            0.739956 -0.200831
        9            0.322761 -0.040201
        10           0.434537 -0.361462
        11           0.410569 -0.239077
        12           0.351185 -0.292620
        13           0.621549 -0.231428
3       0           -0.291921 -0.269673
        1           -0.285731  0.334605
...                       ...       ...
1998    12           0.180254 -0.828056
        13           0.314509 -0.828056
1999    0            2.820385  3.631361
        1            2.602208  1.818528
        2            2.420792  0.678814
        3            2.368358  0.785901
        4            2.576838  0.877690
        5            2.439667  1.091864
        6            2.317997  1.107163
        7            2.965264  1.298390
        8            2.904346  1.780282
        9            2.544015  1.497266
        10           2.503272  1.596704
        11           2.667576  1.673195
        12           2.495702  1.405477
        13           2.911664  1.206601
2000    0           -0.140679  0.311657
        1           -0.119393  0.304008
        2           -0.097721 -0.537391
        3           -0.137423 -0.407357
        4           -0.143456  0.441692
        5           -0.221056 -0.024902
        6           -0.143798  0.143378
        7           -0.236226  0.151027
        8           -0.120636  0.189272
        9           -0.203653 -0.468549
        10          -0.086454 -0.468549
        11          -0.056084  0.418745
        12          -0.143807  0.181623
        13          -0.138221  0.189272

[28000 rows x 2 columns]

In [87]:
scaled_features

{'aqi': [69.10689907646486, 41.82863241974223],
 'month': [6.743497867987794, 3.1398223895350226],
 'pay_cnt': [108.25559585194213, 130.7345540633734],
 'tmp': [20.06643747988851, 9.000948791616628],
 'view_cnt': [15.710891994111511, 42.47333460898241],
 'weather': [12.96204816186815, 29.02202674421276],
 'weekday': [3.0034805507322524, 2.003086129482419],
 'wind': [1.3992824375980313, 2.446529545336942]}

In [88]:
mean, std = scaled_features['pay_cnt']
unnormed_df_result = df_result *std + mean
unnormed_df_result

pred  label
shop_id predict_day                   
1       0            180.341539  205.0
        1            227.889030  198.0
        2            217.560183  181.0
        3            230.814292  236.0
        4            177.601985  204.0
        5            135.267425  228.0
        6            240.909555  215.0
        7            164.235256  240.0
        8            193.162415  159.0
        9            150.812348  196.0
        10           203.698407  155.0
        11           180.230700  156.0
        12           190.931860  175.0
        13           206.162539  195.0
2       0            217.454027  299.0
        1            209.497268  208.0
        2            198.956623   78.0
        3            197.536391   64.0
        4            171.538635   73.0
        5            207.457533   71.0
        6            168.342198  109.0
        7            197.756315  117.0
        8            204.993440   82.0
        9            150.451662  103.0
        10           165.064627   61.0
        11           161.931102   77.0
        12           154.167668   70.0
        13           189.513568   78.0
3       0             70.091494   73.0
        1             70.900690  152.0
...                         ...    ...
1998    12           131.821000    0.0
        13           149.372759    0.0
1999    0            476.977431  583.0
        1            448.454054  346.0
        2            424.736772  197.0
        3            417.881778  211.0
        4            445.137334  223.0
        5            427.204309  251.0
        6            411.297835  253.0
        7            495.918042  278.0
        8            487.954036  341.0
        9            440.846316  304.0
        10           435.519752  317.0
        11           456.999933  327.0
        12           434.530056  292.0
        13           488.910692  266.0
2000    0             89.864013  149.0
        1             92.646814  148.0
        2             95.480094   38.0
        3             90.289678   55.0
        4             89.500997  166.0
        5             79.355921  105.0
        6             89.456245  127.0
        7             77.372663  128.0
        8             92.484333  133.0
        9             81.631048   47.0
        10            96.953040   47.0
        11           100.923522  163.0
        12            89.455033  132.0
        13            90.185336  133.0

[28000 rows x 2 columns]

In [89]:
_pred = unnormed_df_result.pred.values
_label = unnormed_df_result.label.values
loss = np.mean(np.abs((_pred-_label)/(_pred+_label)))
loss

0.21902820985815361

### OUTPUT PREDICT

In [90]:
mean, std = scaled_features['pay_cnt']
unnormed_df_pred = df_pred *std + mean
unnormed_df_pred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,199.636871,255.478760,249.334137,260.140411,209.917542,152.284760,252.599548,197.443115,201.612946,184.073532,226.896545,184.420883,228.415146,232.936676
1,91.424469,89.670135,92.843918,94.503731,88.836456,90.948189,91.666000,85.210526,90.424072,87.908035,93.620811,93.688477,92.066147,93.131378
2,73.393677,70.472366,75.003128,74.003754,74.170593,70.995827,70.759972,69.471970,72.478485,73.904221,78.379868,78.874229,76.778061,75.457771
3,140.629593,140.838989,146.297379,155.330215,115.150024,144.254517,152.707092,133.345825,127.605637,109.881874,145.950409,132.621323,128.479141,140.855103
4,271.380219,259.584412,233.246521,254.704620,225.269730,236.059647,253.201843,230.092133,252.921265,236.527191,205.948029,196.502686,171.322632,226.849518
5,90.015656,84.715576,95.501999,92.457199,85.269333,85.665352,87.242508,80.681915,81.710022,84.883537,97.255440,93.166336,87.376038,91.640495
6,145.696884,150.288300,162.924316,167.605026,134.915649,131.532013,149.978836,133.375519,146.413498,115.355995,161.384323,138.431503,149.457123,149.002243
7,70.251114,64.556328,77.214333,67.053314,67.757835,61.871868,63.448452,62.305485,64.927811,68.395882,78.521683,78.121078,71.459656,75.383148
8,323.575226,302.054230,271.373260,296.009460,266.613281,296.568512,267.138184,300.819458,316.567932,265.423737,252.516083,251.975830,256.362640,310.276703
9,86.508049,86.311218,86.582382,88.638062,85.233711,87.493683,87.923828,82.950645,89.552277,84.776268,86.816956,89.069016,89.407959,89.390564


In [91]:
negtive_check(unnormed_df_pred)
unnormed_df_pred


no negative number~


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,199.636871,255.478760,249.334137,260.140411,209.917542,152.284760,252.599548,197.443115,201.612946,184.073532,226.896545,184.420883,228.415146,232.936676
1,91.424469,89.670135,92.843918,94.503731,88.836456,90.948189,91.666000,85.210526,90.424072,87.908035,93.620811,93.688477,92.066147,93.131378
2,73.393677,70.472366,75.003128,74.003754,74.170593,70.995827,70.759972,69.471970,72.478485,73.904221,78.379868,78.874229,76.778061,75.457771
3,140.629593,140.838989,146.297379,155.330215,115.150024,144.254517,152.707092,133.345825,127.605637,109.881874,145.950409,132.621323,128.479141,140.855103
4,271.380219,259.584412,233.246521,254.704620,225.269730,236.059647,253.201843,230.092133,252.921265,236.527191,205.948029,196.502686,171.322632,226.849518
5,90.015656,84.715576,95.501999,92.457199,85.269333,85.665352,87.242508,80.681915,81.710022,84.883537,97.255440,93.166336,87.376038,91.640495
6,145.696884,150.288300,162.924316,167.605026,134.915649,131.532013,149.978836,133.375519,146.413498,115.355995,161.384323,138.431503,149.457123,149.002243
7,70.251114,64.556328,77.214333,67.053314,67.757835,61.871868,63.448452,62.305485,64.927811,68.395882,78.521683,78.121078,71.459656,75.383148
8,323.575226,302.054230,271.373260,296.009460,266.613281,296.568512,267.138184,300.819458,316.567932,265.423737,252.516083,251.975830,256.362640,310.276703
9,86.508049,86.311218,86.582382,88.638062,85.233711,87.493683,87.923828,82.950645,89.552277,84.776268,86.816956,89.069016,89.407959,89.390564


In [93]:

int_df_pred = unnormed_df_pred.apply(np.round).astype(np.int16)
int_df_pred

int_df_pred.insert(value=range(1, 2001), loc=0, column='id')
int_df_pred

int_df_pred.to_csv('pred/pred_2.csv', header=False, index=False)
int_df_pred

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,200,255,249,260,210,152,253,197,202,184,227,184,228,233
1,2,91,90,93,95,89,91,92,85,90,88,94,94,92,93
2,3,73,70,75,74,74,71,71,69,72,74,78,79,77,75
3,4,141,141,146,155,115,144,153,133,128,110,146,133,128,141
4,5,271,260,233,255,225,236,253,230,253,237,206,197,171,227
5,6,90,85,96,92,85,86,87,81,82,85,97,93,87,92
6,7,146,150,163,168,135,132,150,133,146,115,161,138,149,149
7,8,70,65,77,67,68,62,63,62,65,68,79,78,71,75
8,9,324,302,271,296,267,297,267,301,317,265,253,252,256,310
9,10,87,86,87,89,85,87,88,83,90,85,87,89,89,89


In [99]:
last_half_month = df_usr_view_pay.loc[(slice(None), slice('2016-10-17', '2016-10-31')), ('pay_cnt')]
mean, std = scaled_features['pay_cnt']
last_half_month = last_half_month * std + mean
last_half_month

shop_id  time_stamp
1        2016-10-17    207.0
         2016-10-18    244.0
         2016-10-19    177.0
         2016-10-20    207.0
         2016-10-21    230.0
         2016-10-22    202.0
         2016-10-23    243.0
         2016-10-24    216.0
         2016-10-25    255.0
         2016-10-26    172.0
         2016-10-27    226.0
         2016-10-28    257.0
         2016-10-29    251.0
         2016-10-30    281.0
         2016-10-31    180.0
2        2016-10-17      0.0
         2016-10-18      0.0
         2016-10-19      0.0
         2016-10-20      0.0
         2016-10-21     61.0
         2016-10-22    155.0
         2016-10-23    143.0
         2016-10-24     78.0
         2016-10-25     82.0
         2016-10-26     97.0
         2016-10-27     83.0
         2016-10-28     96.0
         2016-10-29     96.0
         2016-10-30    115.0
         2016-10-31     80.0
                       ...  
1999     2016-10-17    309.0
         2016-10-18    285.0
         2016-10-19    